Имеются данные adult.csv (см. в материалах к занятию).
Целевой переменной является уровень дохода income (крайний правый столбец).
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.
Готовый ноутбук выложить на гитхаб и прислать ссылку.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [29]:
data = pd.read_csv( 'adult.csv' )
data.head(100)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20,Private,479296,HS-grad,9,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,40,United-States,<=50K
96,25,Private,235218,Bachelors,13,Never-married,Exec-managerial,Own-child,White,Female,0,0,40,United-States,<=50K
97,49,Private,164877,10th,6,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,40,United-States,<=50K
98,59,Private,272087,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,>50K


In [77]:
# data.info(), data_new.info()

In [79]:
# for col in data.columns:
#     pct_missing = data[col].isnull().mean()
#     print(f'{col} - {pct_missing :.1%}')

In [75]:
data_new = data[(data.workclass!='?') & (data.occupation!='?') & (data['native-country']!='?')]
data_new

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [ ]:
# X = data_new[['age'], ['gender'], ['hours-per-week']] # двухконтурные скобки, т.к.может быть не один показатель.
# y = data_new['income']

In [126]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data_new[ [ 'age', 'occupation', 'income'] ]
# selectedColumns
# столбец sport является категориальной переменной
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями. one hot incoding
X = pd.get_dummies( selectedColumns, columns = [ 'occupation' ] ) # функция pandas one hot incoding преобразования 
# X
# столбец sex является целевой переменной, удаляем его из X
del X['income']
X.head()

,age,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,25,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,38,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,28,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,44,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,34,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [181]:
# целевая переменная (столбец income) снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [182]:
le.fit( data_new['income'] )

LabelEncoder()

In [183]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [184]:
le.transform( [ '<=50K', '>50K', '<=50K' ] )

array([0, 1, 0], dtype=int64)

In [185]:
y = pd.Series( data = le.transform( data_new['income'] ) )
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [186]:
model = LogisticRegression()

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 

In [188]:
model.fit(X_train, y_train)
predictions = model.predict_proba( X_test )

C:\Users\S.Nikulin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [195]:
predictions[:5]

array([[0.93884504, 0.06115496],
       [0.87582435, 0.12417565],
       [0.72188466, 0.27811534],
       [0.87347922, 0.12652078],
       [0.67023926, 0.32976074]])

In [196]:
model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 1])

In [197]:
model.score(X_train, y_train)

0.7604169957352709

In [198]:
model.score(X_test, y_test)# метод возвращает значение коэффициента детерминации (оценка качества модели: сравниваем со средней моделью)

0.7629542271688656

In [199]:
from sklearn.pipeline import make_pipeline  # поток последовательных действий
from sklearn.preprocessing import StandardScaler

In [200]:
pipe_model = make_pipeline(StandardScaler(),LogisticRegression(max_iter=1000)) 

In [201]:
pipe_model.fit(X_train, y_train)
predictions = pipe_model.predict_proba( X_test )
predictions[:5]

array([[0.93848153, 0.06151847],
       [0.87844882, 0.12155118],
       [0.72271437, 0.27728563],
       [0.87429612, 0.12570388],
       [0.67097208, 0.32902792]])

In [202]:
pipe_model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 1])

In [203]:
pipe_model.score(X_train, y_train)

0.7603854051492656

In [204]:
pipe_model.score(X_test, y_test) # результат аналогичный, что и в методе выше. Разницы нет.

0.7629542271688656

In [205]:
from sklearn.svm import SVC

# метод SVM с маштабированием.
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [206]:
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [207]:
clf.score(X_train, y_train)

0.7737166324435318

In [208]:
clf.score(X_test, y_test) # метод SVM с маштабирования дает нам рез-т выше чем pipeline.

0.7757794648780129

In [209]:
svc = SVC(gamma='auto') 

In [210]:
svc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [211]:
svc.score(X_train, y_train)

0.7752013899857843

In [212]:
svc.score(X_test, y_test) # метод SVM без маштабирования дает нам самый высокий рез-т.

0.7762217144541903